# Partager ses démos avec d'autres

Installez les bibliothèques 🤗 Transformers et 🤗 Gradio pour exécuter ce *notebook*.

In [ ]:
!pip install datasets transformers[sentencepiece]
!pip install gradio

In [ ]:
import gradio as gr

title = "Poser une question (en anglais) à Rick"
description = """
Le bot a été entraîné à répondre à des questions basées sur les dialogues de Rick et Morty (en anglais). Demandez à Rick ce que vous voulez !
<img src="https://huggingface.co/spaces/course-demos/Rick_and_Morty_QA/resolve/main/rick.png" width=200px>
"""

article = "Consultez [le bot original Rick et Morty](https://huggingface.co/spaces/kingabzpro/Rick_and_Morty_Bot) sur lequel cette démo est basée."

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("ericzhou/DialoGPT-Medium-Rick_v2")
model = AutoModelForCausalLM.from_pretrained("ericzhou/DialoGPT-Medium-Rick_v2")

def predict(input, history=[]):
    # tokenizer la nouvelle phrase d'entrée
    new_user_input_ids = tokenizer.encode(input + tokenizer.eos_token, return_tensors='pt')

    # ajouter les nouveaux tokens d'entrée de l'utilisateur à l'historique de chat
    bot_input_ids = torch.cat([torch.LongTensor(history), new_user_input_ids], dim=-1)

    # générer une réponse 
    history = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id).tolist()

    # convertit les tokens en texte, puis divise les réponses dans le bon format.
    response = tokenizer.decode(history[0]).split("<|endoftext|>")
    response = [(response[i], response[i+1]) for i in range(0, len(response)-1, 2)]  # convertir en tuples de liste
    return response, history

gr.Interface(
    fn=predict,
    inputs="textbox",
    outputs="text",
    title=title,
    description=description,
    article=article,
    examples=[["What are you doing?"], ["Where should we time travel to?"]],
).launch()

In [ ]:
# Vous devez récupérer le fichier pytorch_model.bin ici https://huggingface.co/spaces/course-demos/Sketch-Recognition/blob/main/pytorch_model.bin

import torch
import gradio as gr
from torch import nn
import requests
from google.colab import drive
drive.mount('/content/MyDrive/pytorch_model.bin')

LABELS = requests.get("https://huggingface.co/spaces/course-demos/Sketch-Recognition/raw/main/class_names.txt").text.replace("\n","").split("\r")

model = nn.Sequential(
    nn.Conv2d(1, 32, 3, padding="same"),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(32, 64, 3, padding="same"),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(64, 128, 3, padding="same"),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(1152, 256),
    nn.ReLU(),
    nn.Linear(256, len(LABELS)),
)
state_dict = torch.load("pytorch_model.bin", map_location="cpu")
model.load_state_dict(state_dict, strict=False)
model.eval()


def predict(im):
    x = torch.tensor(im, dtype=torch.float32).unsqueeze(0).unsqueeze(0) / 255.0
    with torch.no_grad():
        out = model(x)
    probabilities = torch.nn.functional.softmax(out[0], dim=0)
    values, indices = torch.topk(probabilities, 5)
    return {LABELS[i]: v.item() for i, v in zip(indices, values)}


interface = gr.Interface(
    predict,
    inputs="sketchpad",
    outputs="label",
    theme="huggingface",
    title="Reconnaissance de croquis",
    description="Qui veut jouer au Pictionary ? Dessinez un objet courant comme une pelle ou un ordinateur portable, et l'algorithme le devinera en temps réel !",
    article="<p style='text-align: center'>Reconnaissance de croquis | Modèle de démonstration</p>",
    live=True,
)
interface.launch(share=True)